Inference Code


In [ ]:
import torch
gpu_no = 0

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

device

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
from datasets import load_from_disk
from urllib.parse import urlparse

model_path = './saved_model/'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to(device)

def get_custom_dataset_inf(filename='./Phrase_Banks/test.csv'):

    def map_to_label(label):
        if label == 'neutral':
            return 1
        elif label == 'positive':
            return 2
        elif label == 'negative':
            return 0
        else:       
            return 1
    
    combined_column = 'text'

    def replace_nan(column, value = ""):
        data[column].fillna(value, inplace = True)

    data = pd.read_csv(filename, on_bad_lines='skip',encoding="unicode_escape")
    print(data.columns)

    data['out'] = data[combined_column].astype(str)
    data['out'] = data['out'].astype(str)
    texts = data["out"].tolist()


    return {"texts":texts}


params = get_custom_dataset_inf()

dataset = {
    "texts" : params["texts"],
}


from datasets import Dataset, DatasetDict

ds = Dataset.from_dict(dataset)
print("hi")
dataset_dict = DatasetDict({"unseen": ds})

dataset_dict


def preprocess_data(examples):
    text = examples['texts']
    encoding = tokenizer(text, padding=True, truncation=True)
    return encoding

encoded_dataset = dataset_dict.map(preprocess_data, batched=True, remove_columns=dataset_dict['unseen'].column_names)

def get_model_op(text, tokenizer = tokenizer, model = model):
    label_text = ['postive', 'negative','neutral']
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits

    probabilities = torch.softmax(logits, dim=1)
    print(probabilities)
    predicted_class = logits.argmax().item()
    return predicted_class,label_text[predicted_class], round(probabilities[0][predicted_class].item(), 2)

In [ ]:
def map_to_label(label):
        if label == 'negative':
            return 1
        elif label == 'neutral':
            return 2
        elif label == 'positive':
            return 0
        else:       
            return 1
        

get_model_op("Food")

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

model_specific_path = ''
batch_size = 16
model.config.pad_token_id = model.config.eos_token_id
lr = 1e-5

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=2,  
    early_stopping_threshold=0.02, 
)

args = TrainingArguments(
    output_dir=  'model_pharse_v5',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=512,
    logging_dir= "./output_logs/" + 'logs_phrase_v5',
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
)


In [ ]:
trainer = Trainer(
    model,
    args,
    tokenizer=tokenizer,
)

In [ ]:
results = trainer.predict(encoded_dataset["unseen"])
results

In [ ]:
import numpy as np

def softmax(x):
    exp_x = np.exp(x - np.max(x))  
    return exp_x / exp_x.sum(axis=-1, keepdims=True)

def get_predictions(results, params):
  logits = results.predictions
  predictions = []
  for promt, logit in zip(params["texts"], logits):
    probabilities = softmax(logit)
    predicted_class = logit.argmax().item()
    predicted_score = probabilities[predicted_class]
    predictions.append((promt, predicted_class, predicted_score))

  return predictions


predictions = get_predictions(results, params)

In [ ]:
print(len(data),len(df))
merged = pd.merge(data,df,left_index=True, right_index=True)
print(len(merged))